In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import random
import time
import csv
import os
import datetime

def fill_form_randomly(url, times=10, delay_between_submissions=3, save_responses=True):
    """
    Điền ngẫu nhiên vào Google Form nhiều lần và lưu lại thông tin đã điền
    
    Parameters:
    url (str): Địa chỉ URL của Google Form
    times (int): Số lần điền form
    delay_between_submissions (int): Thời gian chờ giữa các lần gửi form (giây)
    save_responses (bool): Có lưu lại câu trả lời không
    """
    # Cấu hình Chrome
    chrome_options = Options()
    # Bỏ comment dòng dưới nếu muốn chạy ngầm không hiển thị trình duyệt
    # chrome_options.add_argument("--headless")
    
    # Khởi tạo WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # Chuẩn bị file CSV để lưu câu trả lời
    responses_data = []
    csv_filename = f"form_responses_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    
    for i in range(times):
        print(f"Đang điền form lần thứ {i+1}/{times}")
        
        # Mở Google Form
        driver.get(url)
        
        # Chờ trang tải xong
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='radiogroup']"))
        )
        
        # Thu thập thông tin về form
        form_responses = {"submission": i+1}
        
        # Tìm tất cả các câu hỏi trắc nghiệm
        radio_groups = driver.find_elements(By.CSS_SELECTOR, "div[role='radiogroup']")
        print(f"Tìm thấy {len(radio_groups)} câu hỏi trắc nghiệm")
        
        # Điền ngẫu nhiên cho mỗi câu hỏi
        for j, group in enumerate(radio_groups):
            # Tìm tiêu đề của câu hỏi
            try:
                question_element = group.find_element(By.XPATH, "./ancestor::div[contains(@class, 'freebirdFormviewerViewItemsItemItem')]/div[contains(@class, 'freebirdFormviewerViewItemsItemItemHeader')]/div[contains(@class, 'freebirdFormviewerViewItemsItemItemTitleContainer')]")
                question_text = question_element.text.strip()
                if not question_text:
                    question_text = f"Câu hỏi {j+1}"
            except:
                question_text = f"Câu hỏi {j+1}"
            
            # Tìm các lựa chọn trong câu hỏi
            options = group.find_elements(By.CSS_SELECTOR, "div[role='radio']")
            
            if options:
                # Chọn ngẫu nhiên một đáp án
                random_index = random.randrange(len(options))
                random_option = options[random_index]
                
                # Tìm nội dung của đáp án được chọn
                try:
                    option_text = random_option.find_element(By.CSS_SELECTOR, "div.docssharedWizToggleLabeledContent").text.strip()
                    if not option_text:
                        option_text = f"Đáp án {random_index + 1}"
                except:
                    option_text = f"Đáp án {random_index + 1}"
                
                print(f"  {question_text}: Chọn '{option_text}'")
                
                # Lưu thông tin câu trả lời
                form_responses[question_text] = option_text
                
                # Click vào đáp án
                driver.execute_script("arguments[0].scrollIntoView();", random_option)
                random_option.click()
        
        # Thêm vào danh sách responses
        if save_responses:
            responses_data.append(form_responses)
        
        # Tìm và nhấn nút Submit
        try:
            submit_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div[role='button'][jsname='M2UYVd']"))
            )
            driver.execute_script("arguments[0].scrollIntoView();", submit_button)
            submit_button.click()
            
            # Chờ gửi form thành công
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Đã ghi lại phản hồi của bạn') or contains(text(), 'Đã gửi phản hồi của bạn')]"))
            )
            print(f"✓ Đã gửi form thành công lần {i+1}")
            
            # Chờ một chút trước khi gửi form tiếp theo
            time.sleep(delay_between_submissions)
            
        except Exception as e:
            print(f"Lỗi khi gửi form: {e}")
    
    # Lưu dữ liệu vào file CSV
    if save_responses and responses_data:
        # Lấy tất cả các key có thể có
        all_keys = set()
        for resp in responses_data:
            all_keys.update(resp.keys())
        
        fieldnames = ["submission"] + sorted([k for k in all_keys if k != "submission"])
        
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(responses_data)
        
        print(f"Đã lưu thông tin các lần điền form vào file: {os.path.abspath(csv_filename)}")
    
    # Đóng trình duyệt khi hoàn thành
    driver.quit()
    print("Đã hoàn thành việc điền form!")
    
    return csv_filename if save_responses else None




In [3]:
def read_links_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file if line.strip()]

if __name__ == "__main__":
    try:
        form_links = read_links_from_file("links.txt")
        
        for i, form_url in enumerate(form_links, 1):
            print(f"\n==> Đang điền form {i}/{len(form_links)}: {form_url}")
            csv_file = fill_form_randomly(form_url, times=80, delay_between_submissions=2, save_responses=True)
            
            if csv_file:
                print(f"✔️  Đã lưu kết quả vào file: {csv_file}")
            
            time.sleep(2)  # Nghỉ 2s giữa mỗi form
    
    except Exception as e:
        print(f"Lỗi chung: {str(e)}")


==> Đang điền form 1/3: https://docs.google.com/forms/d/1qNuWYhfaS5zv_PH0n0DG6c2bXi7-t3jSibcCXKwH3NQ/edit
Đang điền form lần thứ 1/80
Tìm thấy 5 câu hỏi trắc nghiệm
  Câu hỏi 1: Chọn 'Đáp án 3'
  Câu hỏi 2: Chọn 'Đáp án 5'
  Câu hỏi 3: Chọn 'Đáp án 3'
  Câu hỏi 4: Chọn 'Đáp án 1'
  Câu hỏi 5: Chọn 'Đáp án 4'
Lỗi khi gửi form: Message: 
Stacktrace:
	GetHandleVerifier [0x00BFD363+60275]
	GetHandleVerifier [0x00BFD3A4+60340]
	(No symbol) [0x00A306F3]
	(No symbol) [0x00A78690]
	(No symbol) [0x00A78A2B]
	(No symbol) [0x00AC0EE2]
	(No symbol) [0x00A9D0D4]
	(No symbol) [0x00ABE6EB]
	(No symbol) [0x00A9CE86]
	(No symbol) [0x00A6C623]
	(No symbol) [0x00A6D474]
	GetHandleVerifier [0x00E48FE3+2467827]
	GetHandleVerifier [0x00E445E6+2448886]
	GetHandleVerifier [0x00E5F80C+2560028]
	GetHandleVerifier [0x00C13DF5+153093]
	GetHandleVerifier [0x00C1A3BD+179149]
	GetHandleVerifier [0x00C04BB8+91080]
	GetHandleVerifier [0x00C04D60+91504]
	GetHandleVerifier [0x00BEFA10+4640]
	BaseThreadInitThunk [0x754B